Scrabble
--------

* Calculate the score for a word (you may ignore double/triple letter/word scores).
* Assign seven tiles chosen randomly from the english alphabet to a rack.
* Letters should be distributed based on the English [Scrabble letter distribution](https://en.wikipedia.org/wiki/Scrabble_letter_distributions) (you may ignore blank tiles).
* Find a valid word formed from the seven tiles. A list of valid words can be found in `twl06.txt`.
* Find the longest valid word that can be formed from the seven tiles.
* Find the highest scoring word that can be formed.
* Find the highest scoring word if any one of the letters can score triple.

In [1]:
from collections import defaultdict
import random
import itertools
import urllib, urllib.request
import os

### Prepping the dictionary
Let's download the dictionary as supplied by The Guardian

In [2]:
durl = 'https://raw.githubusercontent.com/guardian/pairing-tests/master/scrabble/twl06.txt'
fn = urllib.parse.urlsplit(durl).path.split('/')[-1]

if not os.path.exists(fn):
    urllib.request.urlretrieve(durl, fn)

In [3]:
%%time
# it's small enough to read at once
with open('twl06.txt') as f:
    lwords = f.read().split('\n') # I'll keep it in a list for now as well
    lwords = list(filter(lambda x: len(x) > 0, lwords)) # looping twice, but it's a small sample, so
    words = set(lwords)

CPU times: user 174 ms, sys: 35.5 ms, total: 210 ms
Wall time: 242 ms


In [4]:
len(words)

178691

Yeah, constant time lookup in sets is gonna be handy.

In [5]:
%timeit 'zymosis' in words

The slowest run took 19.10 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 106 ns per loop


In [6]:
%timeit 'zymosis' in lwords

100 loops, best of 3: 5.03 ms per loop


In order to best match random tile combinations, it will be good to sort all words by their letters. That is, sort letters within a word, so 'letter' becomes 'eelrtt'. That way I know that I can use letters 'eelrtt' to construct 'letter' outright. In constant time, actually.

In [7]:
%%time
skwords = defaultdict(list)
for l in words:
    skwords[''.join(sorted(l))] += [l]

CPU times: user 1.26 s, sys: 67.1 ms, total: 1.33 s
Wall time: 1.6 s


It's a bit slow to generate, but it will save me quite a bit of time later on. At this point the question is - what are we optimising? A cold start? If so, this is not the best way. But, we could easily cache this sorted representation.

### Tiles
As [Wikipedia says](https://en.wikipedia.org/wiki/Scrabble_letter_distributions#English):

*English-language editions of Scrabble contain 100 letter tiles, in the following distribution*:

- 2 blank tiles (scoring 0 points)
- 1 point: E ×12, A ×9, I ×9, O ×8, N ×6, R ×6, T ×6, L ×4, S ×4, U ×4
- 2 points: D ×4, G ×3
- 3 points: B ×2, C ×2, M ×2, P ×2
- 4 points: F ×2, H ×2, V ×2, W ×2, Y ×2
- 5 points: K ×1
- 8 points: J ×1, X ×1
- 10 points: Q ×1, Z ×1

In [8]:
def get_rack(num):
    """Generate a proper Scrabble rack of size `num`"""
    letters = list(12*'e' + 9*'ai' + 8*'o' + 6*'nrt' + 4*'lsud' + 3*'g' + 2*'bcmpfhvwy' + 'kjxqz') # ignoring blanks
    return random.sample(letters, num)

In [9]:
sc = [tuple(j.split('/')) for j in '1/eaionrtlsu, 2/dg, 3/bcmp, 4/fhvwy, 5/k, 8/jx, 10/qz'.split(', ')]
scores = {l: int(k) for k,v in sc for l in list(v)}

In [10]:
def wscore(word):
    """What's the word score with no trebling"""
    return sum([scores[l] for l in word])

In [11]:
rack = get_rack(7)

I only need 128 tries, because the sum of (n choose k) is 2^n and 2^7 = 128

In [12]:
for j in range(8):
    print(len(list(itertools.combinations(''.join(sorted(rack)), j))))

1
7
21
35
35
21
7
1


So once we have the dictionary all set, we can have a quick *warm start* function.

In [13]:
def get_words(rack, scf = wscore):
    res = defaultdict(list)
    for j in range(8):
        for tl in itertools.combinations(''.join(sorted(rack)), j):
            chrs = ''.join(tl)
            if chrs in skwords:
                res[scf(chrs)] += skwords[chrs]
    return res

get_words(get_rack(7))

defaultdict(list,
            {2: ['ae', 'ae', 'oe', 'oe'],
             5: ['aw', 'we', 'we', 'ow', 'wo'],
             6: ['ka',
              'awe',
              'wae',
              'awe',
              'wae',
              'wee',
              'ewe',
              'owe',
              'woe',
              'owe',
              'woe'],
             7: ['kae',
              'kea',
              'kae',
              'kea',
              'oak',
              'oka',
              'koa',
              'eke',
              'eek',
              'oke',
              'oke',
              'awee'],
             8: ['akee'],
             9: ['jo'],
             10: ['jee', 'joe', 'joe', 'wok'],
             11: ['ajee',
              'weka',
              'weak',
              'wake',
              'weka',
              'weak',
              'wake',
              'week',
              'woke',
              'woke'],
             12: ['awoke', 'awoke'],
             13: ['jaw', 'jew', 'jew', 'jo

In [14]:
%timeit get_words(get_rack(7))

1000 loops, best of 3: 271 µs per loop


If I were to think of an algorithm for a cold start, I'd probably do something like this.

In [15]:
def get_cwords(rack, scf = wscore):
    res = defaultdict(list)
    tls = set([''.join(tl) for j in range(8) for tl in itertools.combinations(''.join(sorted(rack)), j)])
    
    for j in words:
        a = ''.join(sorted(j))
        if a in tls:
            res[scf(j)] += [j]

    return res

get_cwords(get_rack(7))

defaultdict(list,
            {2: ['or', 'ut', 'us', 'so', 'os', 'to'],
             3: ['sou',
              'rut',
              'ort',
              'uts',
              'sot',
              'tor',
              'ors',
              'our',
              'rot',
              'out'],
             4: ['sort',
              'sour',
              'rust',
              'tour',
              'orts',
              'ours',
              'tors',
              'rout',
              'oust',
              'rots',
              'ruts',
              'outs'],
             5: ['ow', 'tours', 'torus', 'stour', 'roust', 'wo', 'routs'],
             6: ['wos', 'row', 'two', 'tow', 'sow', 'wot'],
             7: ['kor',
              'tsk',
              'kos',
              'tows',
              'twos',
              'wost',
              'wots',
              'swot',
              'suk',
              'wort',
              'stow',
              'trow',
              'rows'],
             8: ['wurst',

In [16]:
%timeit get_cwords(get_rack(7))

1 loop, best of 3: 633 ms per loop


So that's about 1500 times slower, but that's because we're looping all the words instead of querying the word set in constant time.

### Triple scoring
*Find the highest scoring word if any one of the letters can score triple.*

In [17]:
def wscore3(word):
    """Let's triple the top letter, that's the best we can do"""
    sc = [scores[l] for l in word]
    return sum(sc) + 2*max(sc)

In [18]:
get_words(get_rack(7), wscore3)

defaultdict(list,
            {4: ['la', 'al', 'ar', 'as', 'us'],
             5: ['lar', 'las', 'als', 'sal', 'ras', 'ars', 'sau'],
             6: ['lars', 'saul', 'sura', 'ursa', 'slur'],
             7: ['ad', 'sural'],
             8: ['dal', 'lad', 'rad', 'sad', 'ads', 'urd'],
             9: ['lard',
              'lads',
              'dals',
              'dual',
              'auld',
              'laud',
              'sard',
              'rads',
              'dura',
              'surd',
              'urds'],
             10: ['lards', 'dural', 'duals', 'lauds', 'duras'],
             13: ['ha', 'ah', 'sh', 'uh'],
             14: ['rah', 'ahs', 'ash', 'sha', 'has'],
             15: ['dah',
              'had',
              'duh',
              'harl',
              'lash',
              'haul',
              'hula',
              'rash',
              'hurl',
              'lush',
              'shul',
              'rush',
              'rhus'],
             16: ['dh

In [19]:
get_cwords(get_rack(7), wscore3)

defaultdict(list,
            {4: ['oe'],
             7: ['de', 'od', 'do', 'ed'],
             8: ['doe', 'ode', 'dee'],
             13: ['ow', 'of', 'ef', 'fe', 'we', 'wo'],
             14: ['foe', 'fee', 'owe', 'wee', 'ewe', 'woe'],
             15: ['def', 'dew', 'fed', 'dow', 'wed'],
             16: ['owed', 'weed', 'feod', 'feed'],
             17: ['few'],
             25: ['ex', 'ox'],
             27: ['dex'],
             28: ['exed'],
             29: ['fox'],
             32: ['fedex', 'foxed']})